#### Libraries importeren

In [13]:
import json
import pandas as pd
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

#### Data inladen en naar dataframe omzetten

In [4]:
data = json.load(open('Data/Intent.json'))
df = pd.DataFrame(data['intents'])
df = df[['intent', 'text', 'responses']]
df.head()

,intent,text,responses
0,Greeting,"[Hi, Hi there, Hola, Hello, Hello there, Hya, ...","[Hi human, please tell me your GeniSys user, H..."
1,GreetingResponse,"[My user is Adam, This is Adam, I am Adam, It ...","[Great! Hi <HUMAN>! How can I help?, Good! Hi ..."
2,CourtesyGreeting,"[How are you?, Hi how are you?, Hello how are ...","[Hello, I am great, how are you? Please tell m..."
3,CourtesyGreetingResponse,"[Good thanks! My user is Adam, Good thanks! Th...","[Great! Hi <HUMAN>! How can I help?, Good! Hi ..."
4,CurrentHumanQuery,"[What is my name?, What do you call me?, Who d...","[You are <HUMAN>! How can I help?, Your name i..."


#### Functies maken voor tokenizen

In [5]:
nltk.download('punkt_tab')
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence.lower())

def stem (word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1.0
    return bag

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\boris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#### Woordenlijst maken

In [6]:
all_words = []
tags = []
xy = []

for _, intent in df.iterrows():
    tag = intent["intent"]
    tags.append(tag)
    
    for pattern in intent["text"]:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))
        
all_words = sorted(set(stem(w) for w in all_words if w not in ["?", ".", "!"]))
tags = sorted(set(tags))

#### Traindata samenstellen

In [7]:
X_train = []
Y_train = []

for pattern_sentence, tag in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    Y_train.append(tags.index(tag))
    
X_train = np.array(X_train)
Y_train = np.array(Y_train)
len(all_words)

114

In [35]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = Y_train
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
    
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=8, shuffle=True, num_workers=0)

#### Intent Classifier netwerk bouwen

In [41]:
model = nn.Sequential(
    nn.Linear(len(X_train[0]), 50),
    nn.ReLU(),
    nn.Linear(50, len(tags))
)
print(len(X_train[0]))
print(len(tags))

114
8


#### Loss & optimalisator declareren en check voor cuda

In [37]:
criterium = nn.CrossEntropyLoss()
optimalisator = optim.Adam(model.parameters(), lr=0.001)

component = torch.device("cuda" if torch.cuda.is_available()
                         else "cpu")
model = model.to(component)
print(component)

cpu


In [39]:
num_epochs = 100
for epoch in range(num_epochs):
    for words, labels in train_loader:
                
        words = words.to(component)
        labels = labels.to(component)
        labels = labels.long()
        
        outputs = model(words)
        loss = criterium(outputs, labels)
        
        optimalisator.zero_grad()
        loss.backward()
        optimalisator.step()
        
if (epoch+1) % 100 == 0:
    print(f"Epoch [{epoch + 1} / {num_epochs}], Loss: {loss.item():.4f}")
        

IndexError: Target 15 is out of bounds.